In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Embedding, GlobalMaxPooling1D, Dropout, Bidirectional, LSTM
import feature_builder
import util

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
def prepare_for_cnn(df):
    processed = feature_builder.process_dataset(df)
    return (processed, processed.to_numpy().reshape(processed.shape[0], 1, processed.shape[1]))

In [ ]:
processed, X_train = prepare_for_cnn(train_df)
y_train = train_df['target'].values

In [ ]:
pd.set_option('display.max_colwidth', -1)
processed

In [ ]:
def RNN():
    model = Sequential([
        Bidirectional(LSTM(64, return_sequences=True), input_shape=(X_train.shape[1], X_train.shape[2])),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1)
    ])
    return model

Compilo ni modelo

In [ ]:
model = RNN()
model.summary()
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

Entrenamos

In [ ]:
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=8)
history = model.fit(X_train, y_train, epochs=50, shuffle=False, validation_split=0.33)#, callbacks=[es])
util.plot_history(history)

In [1]:
ids = test_df['id']

processed, final_test = prepare_for_cnn(test_df)
prediction = model.predict_classes(final_test)

final_df = pd.DataFrame({'target': [x[0] for x in prediction]}, index=ids)
final_df

NameError: name 'test_df' is not defined

In [ ]:
final_df.to_csv('lstm.csv')

In [19]:
final_df['target'].value_counts()

0    2317
1    946 
Name: target, dtype: int64